#**Preparation Colab Env**

##Mounting Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


## Import Library

In [ ]:
!pip install geopandas
!pip install descartes
!pip install Fiona 

In [ ]:
import os, sys
from tqdm.notebook import tqdm
import datetime
import geopandas as gpd
import descartes
import fiona
import shapely
import ee
import ee.mapclient
import gdal
import json

##Authenticate & Initialize Earth Engine

In [ ]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=2SsBVLag4TQV3fEG-nuEhg6yN32HmH_5gCdKY10_Bsw&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/0gFHY0Fn9kh5r1EvjsoBDBqeGNTx-LXFmLH1yLap5eCpnV2l93W_J24

Successfully saved authorization token.


# **Define Requirement**

In [ ]:
DIRECTORY = '/gdrive/My Drive/SKRIPSI (MODELLING)'
BATAS_ADMINISTRATIF_PROV = os.path.join(DIRECTORY,'SHP INDONESIA/prov.shp')
TUTUPAN_LAHAN = os.path.join(DIRECTORY,'PETA TUTUPAN LAHAN/STIS.gdb')
LAHAN_BAKU_SAWAH = os.path.join(DIRECTORY,'PETA BAKU SAWAH')
PROV_INTR = os.path.join(DIRECTORY, 'INTR PROV')
PROV_GRID = os.path.join(DIRECTORY, 'GRID PROV')

FOLDER_2013 = 'SKRIPSI [LANDSAT 8 (RGB) 2013] RECT'
FOLDER_2014 = 'SKRIPSI [LANDSAT 8 (RGB) 2014] RECT'
FOLDER_2015 = 'SKRIPSI [LANDSAT 8 (RGB) 2015] RECT'
FOLDER_2016 = 'SKRIPSI [LANDSAT 8 (RGB) 2016] RECT'
FOLDER_2017 = 'SKRIPSI [LANDSAT 8 (RGB) 2017] RECT'
FOLDER_2018 = 'SKRIPSI [LANDSAT 8 (RGB) 2018] RECT'

# **Satellite Imagery (Deteksi Sawah)**

## Load Province Grids SHP

In [ ]:
PROV_GRID_SHP = dict()
for file_prov in tqdm(os.listdir(PROV_GRID)):
  if(file_prov.endswith('.shp')):
    try:
      PROV_GRID_SHP['{}'.format(file_prov.replace('.shp',''))] = gpd.read_file(os.path.join(PROV_GRID, file_prov))
      print(file_prov.replace('.shp',''), 'Load Successfully')
    except: print(file_prov.replace('.shp',''), 'Load Failed')

## Load Province Polygon SHP

In [ ]:
PROV_INTR_SHP = dict()
for file_prov in tqdm(os.listdir(PROV_INTR)):
  if(file_prov.endswith('.shp')):
    try:
      PROV_INTR_SHP['{}'.format(file_prov.replace('.shp',''))] = gpd.read_file(os.path.join(PROV_INTR, file_prov))
      print(file_prov.replace('.shp',''), 'Load Successfully')
    except: print(file_prov.replace('.shp',''), 'Load Failed')

## Cloud & Shadow Mask Function (Landsat 8 SR RGB)

In [ ]:
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask1 = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  mask2 = image.mask().reduce('min')
  mask = mask1.And(mask2)
  return image.updateMask(mask)

## Export Satellite Imagery Function (Landsat 8 SR RGB)


In [ ]:
def export_landsat8_RGB(provinsi, start_date, end_date, tahun, folder):
  from tqdm import tqdm_notebook as tqdm
  prov_shp = PROV_INTR_SHP[provinsi]
  grid_shp = PROV_GRID_SHP[provinsi]


  for page in tqdm(prov_shp.PageName):
    try:
      # processing shp polygon to ee.geometry as bound for clip
      prov_poly = prov_shp[prov_shp.PageName == page].geometry
      grid = grid_shp[grid_shp.PageName == page].geometry
      poly_json = json.loads(prov_poly.to_json())
      grid_json = json.loads(grid.to_json())
      
      # poly_json = json.loads(gpd.GeoSeries([prov_poly]).to_json())
      features_geo = poly_json['features'][0]['geometry']
      grid_geo = grid_json['features'][0]['geometry']

      bound = ee.Geometry.MultiPolygon(features_geo['coordinates'])
      grid_bound = ee.Geometry.MultiPolygon(grid_geo['coordinates'])
      # get features collection & clip image
      collection = (ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
                .filterDate(start_date, end_date).map(maskL8sr))
      image_clip = collection.median().clipToCollection(ee.FeatureCollection(bound))
      print('{}_{}'.format(provinsi, page), 'extract clip success')

    except: print('{}_{}'.format(provinsi, page), 'extract clip failed')

    # define task to extract imagery
    task_config = {
        'scale': 30,  
        'region': grid_bound,
        'fileFormat': 'GeoTIFF',
        'driveFolder': folder
        }
    task = ee.batch.Export.image(image_clip.select(['B4', 'B3', 'B2']), '{}_{}_{}'.format(provinsi, str(page), tahun), task_config)
    task.start()

## Export (2013)

In [ ]:
start_2013 = datetime.datetime(2013, 4, 1)
end_2013 = datetime.datetime(2014, 4, 1)

for provinsi in PROV_GRID_SHP:
  export_landsat8_RGB(provinsi, start_2013, end_2013, '2013', FOLDER_2013)

## Export (2014)

In [ ]:
start_2014 = datetime.datetime(2014, 1, 1)
end_2014 = datetime.datetime(2015, 1, 1)

for provinsi in PROV_GRID_SHP:
  export_landsat8_RGB(provinsi, start_2014, end_2014, '2014', FOLDER_2014)

## Export (2015)

In [ ]:
start_2015 = datetime.datetime(2015, 1, 1)
end_2015 = datetime.datetime(2016, 1, 1)

for provinsi in PROV_GRID_SHP:
  export_landsat8_RGB(provinsi, start_2015, end_2015, '2015', FOLDER_2015)

## Export (2016)

In [ ]:
start_2016 = datetime.datetime(2016, 1, 1)
end_2016 = datetime.datetime(2017, 1, 1)

for provinsi in PROV_GRID_SHP:
  export_landsat8_RGB(provinsi, start_2016, end_2016, '2016', FOLDER_2016)

## Export (2017)

In [ ]:
start_2017 = datetime.datetime(2017, 1, 1)
end_2017 = datetime.datetime(2018, 1, 1)

for provinsi in PROV_GRID_SHP:
  export_landsat8_RGB(provinsi, start_2017, end_2017, '2017', FOLDER_2017)

## Export (2018)

In [ ]:
start_2018 = datetime.datetime(2018, 1, 1)
end_2018 = datetime.datetime(2019, 1, 1)

for provinsi in PROV_GRID_SHP:
  export_landsat8_RGB(provinsi, start_2018, end_2018, '2018', FOLDER_2018)

# **Satellite Imagery (Tutupan Lahan)**

## Define Directories

In [ ]:
TL_SHP_DIR = os.path.join(DIRECTORY, 'SHP TUTUPAN LAHAN')
HUTAN_LKP = '2001'
HUTAN_LKS = '2002'
HUTAN_MP = '2004'
HUTAN_MS = '20041'
HUTAN_RP = '2005'
HUTAN_RS = '20051'
HUTAN_T = '2006'
SEMAK = '2007'
SEMAK_R = '20071'
PERTANIAN_LKP = '20091'
PERTANIAN_LKS = '20092'
BADAN_AIR = '5001'
BADAN_AIR_R = '50011'
PEMUKIMAN = '2012'
TERBANGUN = '20121'

FOLDER_HUTAN = 'NEW SKRIPSI IMAGE [HUTAN]' 
FOLDER_HUTAN_LK = 'NEW SKRIPSI IMAGE [HUTAN_LK]' 
FOLDER_HUTAN_M = 'NEW SKRIPSI IMAGE [HUTAN_M]' 
FOLDER_HUTAN_R = 'NEW SKRIPSI IMAGE [HUTAN_R]' 
FOLDER_HUTAN_T = 'NEW SKRIPSI IMAGE [HUTAN_T]' 
FOLDER_SEMAK = 'NEW SKRIPSI IMAGE [SEMAK]' 
FOLDER_PERTANIAN_LK = 'NEW SKRIPSI IMAGE [PERTANIAN_LK]' 
FOLDER_BADAN_AIR = 'NEW SKRIPSI IMAGE [BADAN AIR]' 
FOLDER_PEMUKIMAN = 'NEW SKRIPSI IMAGE [PEMUKIMAN]' 

## Load Grid Tutupan Lahan SHP

In [ ]:
TL_GRID_SHP = dict()
for file_grid in tqdm(os.listdir(TL_SHP_DIR)):
  if(file_grid.endswith('.shp')):
    try:
      TL_GRID_SHP['{}'.format(file_grid.replace('.shp',''))] = gpd.read_file(os.path.join(TL_SHP_DIR, file_grid))
      print(file_grid.replace('.shp',''), 'Load Successfully')
    except: print(file_grid.replace('.shp',''), 'Load Failed')

## Cloud Mask Function (Sentinel 2 RGB)

In [ ]:
def maskS2clouds(image):
  cloudBitMask  = ee.Number(2).pow(10).int()
  cirrusBitMask = ee.Number(2).pow(11).int()
  qa = image.select('QA60')
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
    qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask).divide(10000)

## Export Satellite Imagery Function (Sentinel 2 RGB)

In [ ]:
def export_sentinel2_RGB(kode_lahan, start_date, end_date, n, folder):
  from tqdm import tqdm_notebook as tqdm
  grid_shp = TL_GRID_SHP[kode_lahan]


  for page in tqdm(grid_shp.sample(frac = 1).PageName[:n]):
    try:
      grid = grid_shp[grid_shp.PageName == page].geometry
      grid_json = json.loads(grid.to_json())
      grid_geo = grid_json['features'][0]['geometry']
      grid_bound = ee.Geometry.MultiPolygon(grid_geo['coordinates'])

      collection = (ee.ImageCollection('COPERNICUS/S2_SR')
                .filterDate(start_date, end_date).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).map(maskS2clouds))
      
      image_clip = collection.median().clipToCollection(ee.FeatureCollection(grid_bound))
      print('{}_{}'.format(kode_lahan, page), 'extract clip success')
    except: print('{}_{}'.format(kode_lahan, page), 'extract clip failed')

    task_config = {
        'scale': 10,  
        'region': grid_bound,
        'fileFormat': 'GeoTIFF',
        'driveFolder': folder,
        }
    task = ee.batch.Export.image(image_clip.select(['B4', 'B3', 'B2']), '{}_{}'.format(kode_lahan, page), task_config)
    task.start()

## Export Tutupan Lahan

In [ ]:
start = datetime.datetime(2017, 4, 1)
end = datetime.datetime(2018, 12, 31)

### Hutan

In [ ]:
export_sentinel2_RGB(HUTAN_LKS, start, end, 500, FOLDER_HUTAN_LK)
export_sentinel2_RGB(HUTAN_LKP, start, end, 500, FOLDER_HUTAN_LK)

In [ ]:
export_sentinel2_RGB(HUTAN_MP, start, end, 500, FOLDER_HUTAN_M)
export_sentinel2_RGB(HUTAN_MS, start, end, 500, FOLDER_HUTAN_M)

In [ ]:
export_sentinel2_RGB(HUTAN_RP, start, end, 500, FOLDER_HUTAN_R)
export_sentinel2_RGB(HUTAN_RS, start, end, 500, FOLDER_HUTAN_R)

In [ ]:
export_sentinel2_RGB(HUTAN_T, start, end, 500, FOLDER_HUTAN_T)

### Semak

In [ ]:
export_sentinel2_RGB(SEMAK, start, end, 1500, FOLDER_SEMAK)
export_sentinel2_RGB(SEMAK_R, start, end, 1500, FOLDER_SEMAK)

### Pertanian Lahan Kering

In [ ]:
export_sentinel2_RGB(PERTANIAN_LKP, start, end, 1500, FOLDER_PERTANIAN_LK)
export_sentinel2_RGB(PERTANIAN_LKS, start, end, 1500, FOLDER_PERTANIAN_LK)

### Badan Air

In [ ]:
export_sentinel2_RGB(BADAN_AIR, start, end, 1500, FOLDER_BADAN_AIR)
export_sentinel2_RGB(BADAN_AIR_R, start, end, 1500, FOLDER_BADAN_AIR)

### Pemukiman / Lahan Terbangun

In [ ]:
export_sentinel2_RGB(PEMUKIMAN, start, end, 1500, FOLDER_PEMUKIMAN)
export_sentinel2_RGB(TERBANGUN, start, end, 1500, FOLDER_PEMUKIMAN)

# **Satellite Imagery Grid Substract Studi Kasus (Bali)**

## Define Directories

In [ ]:
GRID_SUBSTRACT_BALI = os.path.join(DIRECTORY, 'GRID ALIH FUNGSI BALI')

FOLDER_2013_2014 = 'SUBSTRACT SAWAH BALI 2013-2014'
FOLDER_2014_2015 = 'SUBSTRACT SAWAH BALI 2014-2015'
FOLDER_2015_2016 = 'SUBSTRACT SAWAH BALI 2015-2016'
FOLDER_2016_2017 = 'SUBSTRACT SAWAH BALI 2016-2017'
FOLDER_2017_2018 = 'SUBSTRACT SAWAH BALI 2017-2018'
FOLDER_2013_2018 = 'SUBSTRACT SAWAH BALI 2013-2018'

## Load Grid Substract SHP

In [ ]:
GRID_SUBSTRACT_BALI_SHP = dict()
for file_grid in tqdm(os.listdir(GRID_SUBSTRACT_BALI)):
  if(file_grid.endswith('.shp')):
    try:
      GRID_SUBSTRACT_BALI_SHP['{}'.format(file_grid.replace('.shp',''))] = gpd.read_file(os.path.join(GRID_SUBSTRACT_BALI, file_grid))
      print(file_grid.replace('.shp',''), 'Load Successfully')
    except: print(file_grid.replace('.shp',''), 'Load Failed')

## Cloud Mask Function (Sentinel 2 RGB)

In [ ]:
def maskS2clouds(image):
  cloudBitMask  = ee.Number(2).pow(10).int()
  cirrusBitMask = ee.Number(2).pow(11).int()
  qa = image.select('QA60')
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
    qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask).divide(10000)

## Export Satellite Imagery Function (Sentinel 2 RGB) - Grid Substract

In [ ]:
def export_sentinel2_RGB_SB(periode, start_date, end_date, folder):
  grid_shp = GRID_SUBSTRACT_BALI_SHP[periode]


  for page in tqdm(grid_shp.PageName):
    try:
      grid = grid_shp[grid_shp.PageName == page].geometry
      grid_json = json.loads(grid.to_json())
      grid_geo = grid_json['features'][0]['geometry']
      grid_bound = ee.Geometry.MultiPolygon(grid_geo['coordinates'])

      collection = (ee.ImageCollection('COPERNICUS/S2')
                .filterDate(start_date, end_date).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).map(maskS2clouds))
      
      image_clip = collection.median().clipToCollection(ee.FeatureCollection(grid_bound))
      print('{}_{}'.format(periode, page), 'extract clip success')
    except: print('{}_{}'.format(periode, page), 'extract clip failed')

    task_config = {
        'scale': 10,  
        'region': grid_bound,
        'fileFormat': 'GeoTIFF',
        'driveFolder': folder,
        }
    task = ee.batch.Export.image(image_clip.select(['B4', 'B3', 'B2']), '{}_{}'.format(periode, page), task_config)
    task.start()

## Export Satellite Imagery Function (Landsat 8 RGB) - Grid Substract

In [ ]:
def export_landsat8_RGB_TL(periode, start_date, end_date, folder):
  grid_shp = TL_GRID_SHP[periode]


  for page in tqdm(grid_shp.PageName):
    try:
      grid = grid_shp[grid_shp.PageName == page].geometry
      grid_json = json.loads(grid.to_json())
      grid_geo = grid_json['features'][0]['geometry']

      grid_bound = ee.Geometry.MultiPolygon(grid_geo['coordinates'])
      collection = (ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
                .filterDate(start_date, end_date).map(maskL8sr))
      
      image_clip = collection.mosaic().clipToCollection(ee.FeatureCollection(grid_bound))
      print('{}_{}'.format(periode, page), 'extract clip success')
    except: print('{}_{}'.format(periode, page), 'extract clip failed')

    task_config = {
        'scale': 30,  
        'region': grid_bound,
        'fileFormat': 'GeoTIFF',
        'driveFolder': folder,
        }
    task = ee.batch.Export.image(image_clip.select(['B4', 'B3', 'B2']), '{}_{}'.format(periode, page), task_config)
    task.start()

## Export Every Period

### Export Substract 2013-2018

In [ ]:
start_2018 = datetime.datetime(2018, 1, 1)
end_2018 = datetime.datetime(2019, 1, 1)
export_sentinel2_RGB_SB('2013_2018', start_2018, end_2018, FOLDER_2013_2018)

### Export Substract 2013-2014

In [ ]:
start_2014 = datetime.datetime(2014, 1, 1)
end_2014 = datetime.datetime(2015, 1, 1)
export_landsat8_RGB_TL('2013_2014', start_2014, end_2014, FOLDER_2013_2014)

### Export Substract 2014-2015

In [ ]:
start_2015 = datetime.datetime(2015, 7, 1)
end_2015 = datetime.datetime(2016, 1, 1)
export_sentinel2_RGB_SB('2014_2015', start_2015, end_2015, FOLDER_2014_2015)

### Export Substract 2015-2016

In [ ]:
start_2016 = datetime.datetime(2016, 1, 1)
end_2016 = datetime.datetime(2017, 1, 1)
export_sentinel2_RGB_SB('2015_2016', start_2016, end_2016, FOLDER_2015_2016)

### Export Substract 2016-2017

In [ ]:
start_2017 = datetime.datetime(2017, 1, 1)
end_2017 = datetime.datetime(2018, 1, 1)
export_sentinel2_RGB_SB('2016_2017', start_2017, end_2017, FOLDER_2016_2017)

### Export Substract 2017-2018

In [ ]:
start_2018 = datetime.datetime(2018, 1, 1)
end_2018 = datetime.datetime(2019, 1, 1)
export_sentinel2_RGB_SB('2017_2018', start_2018, end_2018, FOLDER_2017_2018)